In [1]:
import requests
import os
import json
from dotenv import load_dotenv
load_dotenv()

# --- 1. FRED API 설정 정보 ---

# 데이터 시리즈 ID (미국 금리 핵심 지표)
SERIES_IDS = {
    "기준금리 (FEDFUNDS)": "FEDFUNDS",  # 실효 연방기금 금리 (단기 정책 금리)
    "10년물 국채 수익률 (DGS10)": "DGS10",  # 장기 시장 금리 (시장 벤치마크)
}

# 요청 파라미터 설정
# - observation_start: 데이터 시작 날짜 (YYYY-MM-DD)
# - file_type: 응답 데이터 형식 (json이 분석에 용이)
# - frequency: 데이터 주기 (d: 일별, m: 월별, q: 분기별, a: 연간)
REQUEST_PARAMS = {
    # 💡 2015년 10월 1일부터 요청 (약 10년치)
    "observation_start": "2015-10-01", 
    "file_type": "json",
    "api_key" : os.getenv("FRED_API_KEY"), # 환경 변수에서 FRED_API_KEY를 가져와 설정
    "frequency": "m"  # 월별 데이터 요청
}

def get_fred_data(series_id, series_name):
    """
    FRED API에서 특정 시리즈 ID의 데이터를 가져오는 함수
    """
    # FRED API의 기본 URL
    base_url = "https://api.stlouisfed.org/fred/series/observations"
    
    # 시리즈 ID 추가
    params = REQUEST_PARAMS.copy()
    params["series_id"] = series_id

    try:
        response = requests.get(base_url, params=params)
        response.raise_for_status() # HTTP 오류가 발생하면 예외 발생
        
        data = response.json()
        
        print(f"\n--- {series_name} 데이터 ({series_id}) ---")
        
        # 데이터가 있는지 확인
        observations = data.get("observations")
        if not observations:
            print("데이터를 찾을 수 없거나 API 키가 잘못되었습니다.")
            return

        # 💡 최근 12개월 (1년치) 데이터만 출력합니다. (API는 10년치를 가져옴)
        print("최근 12개월 데이터 출력:")
        for obs in observations[-12:]:
            print(f"날짜: {obs['date']}, 금리: {obs['value']}%")

    except requests.exceptions.RequestException as e:
        print(f"API 요청 중 오류 발생: {e}")
    except json.JSONDecodeError:
        print("JSON 디코딩 오류: 응답 형식이 잘못되었습니다.")

# --- 메인 실행 ---
if __name__ == "__main__":
    for name, series_id in SERIES_IDS.items():
        get_fred_data(series_id, name)


--- 기준금리 (FEDFUNDS) 데이터 (FEDFUNDS) ---
최근 12개월 데이터 출력:
날짜: 2024-10-01, 금리: 4.83%
날짜: 2024-11-01, 금리: 4.64%
날짜: 2024-12-01, 금리: 4.48%
날짜: 2025-01-01, 금리: 4.33%
날짜: 2025-02-01, 금리: 4.33%
날짜: 2025-03-01, 금리: 4.33%
날짜: 2025-04-01, 금리: 4.33%
날짜: 2025-05-01, 금리: 4.33%
날짜: 2025-06-01, 금리: 4.33%
날짜: 2025-07-01, 금리: 4.33%
날짜: 2025-08-01, 금리: 4.33%
날짜: 2025-09-01, 금리: 4.22%

--- 10년물 국채 수익률 (DGS10) 데이터 (DGS10) ---
최근 12개월 데이터 출력:
날짜: 2024-11-01, 금리: 4.36%
날짜: 2024-12-01, 금리: 4.39%
날짜: 2025-01-01, 금리: 4.63%
날짜: 2025-02-01, 금리: 4.45%
날짜: 2025-03-01, 금리: 4.28%
날짜: 2025-04-01, 금리: 4.28%
날짜: 2025-05-01, 금리: 4.42%
날짜: 2025-06-01, 금리: 4.38%
날짜: 2025-07-01, 금리: 4.39%
날짜: 2025-08-01, 금리: 4.26%
날짜: 2025-09-01, 금리: 4.12%
날짜: 2025-10-01, 금리: 4.06%


In [2]:
import requests
import os
from dotenv import load_dotenv
import pandas as pd
from datetime import datetime

load_dotenv()

FRED_API_KEY = os.getenv("FRED_API_KEY")
ECOS_API_KEY = os.getenv("ECOS_API_KEY")

# --- FRED 데이터 가져오기 (미국) ---
def get_fred_data(series_id, start_date="2015-01-01"):
    """FRED API에서 데이터 가져오기"""
    base_url = "https://api.stlouisfed.org/fred/series/observations"
    
    params = {
        "series_id": series_id,
        "api_key": FRED_API_KEY,
        "file_type": "json",
        "observation_start": start_date,
        "frequency": "m"
    }
    
    try:
        response = requests.get(base_url, params=params)
        response.raise_for_status()
        data = response.json()
        
        observations = data.get("observations", [])
        df = pd.DataFrame(observations)
        df = df[df['value'] != '.']
        df['date'] = pd.to_datetime(df['date'])
        df['value'] = pd.to_numeric(df['value'])
        
        return df[['date', 'value']]
    
    except Exception as e:
        print(f"FRED API 오류 ({series_id}): {e}")
        return None

# --- 한국은행 데이터 가져오기 ---
def get_ecos_data(start_date="201501"):
    """한국은행 API에서 기준금리 가져오기"""
    
    if not ECOS_API_KEY:
        print("⚠️ ECOS_API_KEY가 설정되지 않았습니다.")
        print("한국은행 API 키를 발급받아 .env 파일에 추가하세요.")
        print("발급 URL: https://ecos.bok.or.kr/")
        return None
    
    print(f"🔍 한국은행 API 호출 중... (API Key: {ECOS_API_KEY[:10]}...)")
    
    base_url = "https://ecos.bok.or.kr/api/StatisticSearch"
    
    params = {
        "service": ECOS_API_KEY,
        "lang": "kr",
        "format": "json",
        "start_count": "1",
        "end_count": "1000",
        "stat_code": "722Y001",
        "cycle_type": "M",
        "start_time": start_date,
        "end_time": datetime.now().strftime("%Y%m"),
        "item_1": "0101000"
    }
    
    # URL 출력 (디버깅용)
    url = f"{base_url}/{ECOS_API_KEY}/json/kr/1/1000/722Y001/M/{start_date}/{datetime.now().strftime('%Y%m')}/0101000"
    print(f"📡 요청 URL: {url}")
    
    try:
        # 한국은행 API는 URL 파라미터 방식이 다릅니다
        url = f"{base_url}/{ECOS_API_KEY}/json/kr/1/1000/722Y001/M/{start_date}/{datetime.now().strftime('%Y%m')}/0101000"
        response = requests.get(url)
        response.raise_for_status()
        data = response.json()
        
        print(f"✅ API 응답 받음")
        print(f"응답 데이터 구조: {list(data.keys())}")
        
        if 'StatisticSearch' in data:
            if 'row' in data['StatisticSearch']:
                rows = data['StatisticSearch']['row']
                print(f"✅ {len(rows)}개 데이터 수신")
                
                df = pd.DataFrame(rows)
                df['date'] = pd.to_datetime(df['TIME'], format='%Y%m')
                df['value'] = pd.to_numeric(df['DATA_VALUE'])
                
                print(f"✅ 데이터 파싱 완료 (첫 데이터: {df.iloc[0]['date']}, 마지막: {df.iloc[-1]['date']})")
                
                return df[['date', 'value']]
            else:
                print(f"❌ 'row' 키가 없음: {data['StatisticSearch']}")
        else:
            print(f"❌ API 오류 응답: {data}")
            
        return None
    
    except Exception as e:
        print(f"❌ 한국은행 API 오류: {e}")
        if 'response' in locals():
            print(f"응답 내용: {response.text[:500]}")
        return None

def display_comparison():
    """한미 금리 비교 출력"""
    print("\n" + "="*80)
    print("📊 한국 vs 미국 금리 비교 (최근 10년, 월별)")
    print("="*80 + "\n")
    
    print("데이터 수집 중...\n")
    
    # 데이터 수집
    kr_rate = get_ecos_data("201501")
    us_fedfunds = get_fred_data("FEDFUNDS", "2015-01-01")
    us_10y = get_fred_data("DGS10", "2015-01-01")
    
    # 데이터 병합
    merged = None
    
    if kr_rate is not None:
        print(f"✅ 한국 데이터 {len(kr_rate)}개 수집 완료")
        kr_rate = kr_rate.rename(columns={'value': '한국_기준금리'})
        merged = kr_rate
    else:
        print("❌ 한국 데이터 수집 실패")
    
    if us_fedfunds is not None:
        print(f"✅ 미국 기준금리 {len(us_fedfunds)}개 수집 완료")
        us_fedfunds = us_fedfunds.rename(columns={'value': '미국_기준금리'})
        if merged is None:
            merged = us_fedfunds
        else:
            merged = pd.merge(merged, us_fedfunds, on='date', how='outer')
    
    if us_10y is not None:
        print(f"✅ 미국 10년물 {len(us_10y)}개 수집 완료")
        us_10y = us_10y.rename(columns={'value': '미국_10년물국채'})
        if merged is None:
            merged = us_10y
        else:
            merged = pd.merge(merged, us_10y, on='date', how='outer')
    
    if merged is None:
        print("❌ 데이터를 가져오지 못했습니다.")
        return
    
    # 날짜순 정렬
    merged = merged.sort_values('date')
    merged['연월'] = merged['date'].dt.strftime('%Y년 %m월')
    
    # 최근 24개월 출력
    print("\n📅 최근 24개월 데이터:\n")
    recent = merged.tail(24)
    
    print(f"{'연월':<15} {'한국 기준금리':<15} {'미국 기준금리':<15} {'미국 10년물':<15}")
    print("-" * 70)
    
    for _, row in recent.iterrows():
        kr = f"{row.get('한국_기준금리', 0):.2f}%" if pd.notna(row.get('한국_기준금리')) else "N/A"
        us_ff = f"{row.get('미국_기준금리', 0):.2f}%" if pd.notna(row.get('미국_기준금리')) else "N/A"
        us_10y_val = f"{row.get('미국_10년물국채', 0):.2f}%" if pd.notna(row.get('미국_10년물국채')) else "N/A"
        
        print(f"{row['연월']:<15} {kr:<15} {us_ff:<15} {us_10y_val:<15}")
    
    # CSV 저장
    output_file = "금리비교_10년.csv"
    merged.to_csv(output_file, index=False, encoding='utf-8-sig')
    print(f"\n💾 전체 데이터가 '{output_file}'에 저장되었습니다.")
    
    # 통계 요약
    print("\n📈 통계 요약 (최근 10년):")
    print("-" * 70)
    for col in ['한국_기준금리', '미국_기준금리', '미국_10년물국채']:
        if col in merged.columns:
            valid_data = merged[col].dropna()
            if len(valid_data) > 0:
                print(f"\n{col}:")
                print(f"  평균: {valid_data.mean():.2f}%")
                print(f"  최고: {valid_data.max():.2f}%")
                print(f"  최저: {valid_data.min():.2f}%")

if __name__ == "__main__":
    display_comparison()


📊 한국 vs 미국 금리 비교 (최근 10년, 월별)

데이터 수집 중...

🔍 한국은행 API 호출 중... (API Key: 5R7Q6RTQNX...)
📡 요청 URL: https://ecos.bok.or.kr/api/StatisticSearch/5R7Q6RTQNXYZJO92A9F1/json/kr/1/1000/722Y001/M/201501/202510/0101000
✅ API 응답 받음
응답 데이터 구조: ['StatisticSearch']
✅ 129개 데이터 수신
✅ 데이터 파싱 완료 (첫 데이터: 2015-01-01 00:00:00, 마지막: 2025-09-01 00:00:00)
✅ 한국 데이터 129개 수집 완료
✅ 미국 기준금리 129개 수집 완료
✅ 미국 10년물 130개 수집 완료

📅 최근 24개월 데이터:

연월              한국 기준금리         미국 기준금리         미국 10년물        
----------------------------------------------------------------------
2023년 11월       3.50%           5.33%           4.50%          
2023년 12월       3.50%           5.33%           4.02%          
2024년 01월       3.50%           5.33%           4.06%          
2024년 02월       3.50%           5.33%           4.21%          
2024년 03월       3.50%           5.33%           4.21%          
2024년 04월       3.50%           5.33%           4.54%          
2024년 05월       3.50%           5.33%           4.48%          
2024

In [3]:
import requests
import os
from dotenv import load_dotenv
import pandas as pd
from datetime import datetime

load_dotenv()

FRED_API_KEY = os.getenv("FRED_API_KEY")
ECOS_API_KEY = os.getenv("ECOS_API_KEY")

# --- FRED 데이터 가져오기 (미국) ---
def get_fred_data(series_id, start_date="2015-01-01"):
    """FRED API에서 데이터 가져오기"""
    base_url = "https://api.stlouisfed.org/fred/series/observations"
    
    params = {
        "series_id": series_id,
        "api_key": FRED_API_KEY,
        "file_type": "json",
        "observation_start": start_date,
        "frequency": "m"
    }
    
    try:
        response = requests.get(base_url, params=params)
        response.raise_for_status()
        data = response.json()
        
        observations = data.get("observations", [])
        df = pd.DataFrame(observations)
        df = df[df['value'] != '.']
        df['date'] = pd.to_datetime(df['date'])
        df['value'] = pd.to_numeric(df['value'])
        
        return df[['date', 'value']]
    
    except Exception as e:
        print(f"FRED API 오류 ({series_id}): {e}")
        return None

# --- 한국은행 데이터 가져오기 ---
def get_ecos_data(start_date="201501"):
    """한국은행 API에서 기준금리 가져오기"""
    
    if not ECOS_API_KEY:
        print("⚠️ ECOS_API_KEY가 설정되지 않았습니다.")
        print("한국은행 API 키를 발급받아 .env 파일에 추가하세요.")
        print("발급 URL: https://ecos.bok.or.kr/")
        return None
    
    print(f"🔍 한국은행 API 호출 중...")
    
    base_url = "https://ecos.bok.or.kr/api/StatisticSearch"
    
    try:
        # 한국은행 API는 이런 형식입니다
        url = f"{base_url}/{ECOS_API_KEY}/json/kr/1/1000/722Y001/M/{start_date}/{datetime.now().strftime('%Y%m')}/0101000"
        response = requests.get(url)
        response.raise_for_status()
        data = response.json()
        
        print(f"✅ API 응답 받음")
        
        if 'StatisticSearch' in data:
            if 'row' in data['StatisticSearch']:
                rows = data['StatisticSearch']['row']
                print(f"✅ {len(rows)}개 데이터 수신")
                
                df = pd.DataFrame(rows)
                df['date'] = pd.to_datetime(df['TIME'], format='%Y%m')
                df['value'] = pd.to_numeric(df['DATA_VALUE'])
                
                return df[['date', 'value']]
            else:
                print(f"❌ 'row' 키가 없음")
        else:
            print(f"❌ API 오류 응답: {data}")
            
        return None
    
    except Exception as e:
        print(f"❌ 한국은행 API 오류: {e}")
        return None

def display_comparison():
    """한미 금리 비교 출력"""
    print("\n" + "="*80)
    print("📊 한국 vs 미국 금리 비교 (최근 10년, 월별)")
    print("="*80 + "\n")
    
    print("데이터 수집 중...\n")
    
    # 데이터 수집
    kr_rate = get_ecos_data("201501")
    us_fedfunds = get_fred_data("FEDFUNDS", "2015-01-01")
    us_10y = get_fred_data("DGS10", "2015-01-01")
    
    # 데이터 병합
    merged = None
    
    if kr_rate is not None:
        print(f"✅ 한국 데이터 {len(kr_rate)}개 수집 완료")
        kr_rate = kr_rate.rename(columns={'value': '한국_기준금리'})
        merged = kr_rate
    else:
        print("❌ 한국 데이터 수집 실패")
    
    if us_fedfunds is not None:
        print(f"✅ 미국 기준금리 {len(us_fedfunds)}개 수집 완료")
        us_fedfunds = us_fedfunds.rename(columns={'value': '미국_기준금리'})
        if merged is None:
            merged = us_fedfunds
        else:
            merged = pd.merge(merged, us_fedfunds, on='date', how='outer')
    
    if us_10y is not None:
        print(f"✅ 미국 10년물 {len(us_10y)}개 수집 완료")
        us_10y = us_10y.rename(columns={'value': '미국_10년물국채'})
        if merged is None:
            merged = us_10y
        else:
            merged = pd.merge(merged, us_10y, on='date', how='outer')
    
    if merged is None:
        print("❌ 데이터를 가져오지 못했습니다.")
        return
    
    # 날짜순 정렬
    merged = merged.sort_values('date')
    merged['연월'] = merged['date'].dt.strftime('%Y년 %m월')
    
    print("\n" + "="*80)
    print("📅 전체 10년 데이터 출력 (2015년 1월 ~ 현재)")
    print("="*80 + "\n")
    
    print(f"{'연월':<15} {'한국 기준금리':<15} {'미국 기준금리':<15} {'미국 10년물':<15}")
    print("-" * 70)
    
    # 전체 데이터 출력
    for _, row in merged.iterrows():
        kr = f"{row.get('한국_기준금리', 0):.2f}%" if pd.notna(row.get('한국_기준금리')) else "N/A"
        us_ff = f"{row.get('미국_기준금리', 0):.2f}%" if pd.notna(row.get('미국_기준금리')) else "N/A"
        us_10y_val = f"{row.get('미국_10년물국채', 0):.2f}%" if pd.notna(row.get('미국_10년물국채')) else "N/A"
        
        print(f"{row['연월']:<15} {kr:<15} {us_ff:<15} {us_10y_val:<15}")
    
    # CSV 저장
    output_file = "금리비교_10년.csv"
    merged.to_csv(output_file, index=False, encoding='utf-8-sig')
    print(f"\n💾 전체 데이터가 '{output_file}'에 저장되었습니다.")
    
    # 통계 요약
    print("\n" + "="*80)
    print("📈 통계 요약 (최근 10년)")
    print("="*80)
    for col in ['한국_기준금리', '미국_기준금리', '미국_10년물국채']:
        if col in merged.columns:
            valid_data = merged[col].dropna()
            if len(valid_data) > 0:
                print(f"\n{col}:")
                print(f"  데이터 개수: {len(valid_data)}개월")
                print(f"  평균: {valid_data.mean():.2f}%")
                print(f"  최고: {valid_data.max():.2f}% ({merged[merged[col] == valid_data.max()]['연월'].values[0]})")
                print(f"  최저: {valid_data.min():.2f}% ({merged[merged[col] == valid_data.min()]['연월'].values[0]})")
                print(f"  현재: {valid_data.iloc[-1]:.2f}%")

if __name__ == "__main__":
    display_comparison()


📊 한국 vs 미국 금리 비교 (최근 10년, 월별)

데이터 수집 중...

🔍 한국은행 API 호출 중...
✅ API 응답 받음
✅ 129개 데이터 수신
✅ 한국 데이터 129개 수집 완료
✅ 미국 기준금리 129개 수집 완료
✅ 미국 10년물 130개 수집 완료

📅 전체 10년 데이터 출력 (2015년 1월 ~ 현재)

연월              한국 기준금리         미국 기준금리         미국 10년물        
----------------------------------------------------------------------
2015년 01월       2.00%           0.11%           1.88%          
2015년 02월       2.00%           0.11%           1.98%          
2015년 03월       1.75%           0.11%           2.04%          
2015년 04월       1.75%           0.12%           1.94%          
2015년 05월       1.75%           0.12%           2.20%          
2015년 06월       1.50%           0.13%           2.36%          
2015년 07월       1.50%           0.13%           2.32%          
2015년 08월       1.50%           0.14%           2.17%          
2015년 09월       1.50%           0.14%           2.17%          
2015년 10월       1.50%           0.12%           2.07%          
2015년 11월       1.50%           0.12%  

In [4]:
import requests
import os
from dotenv import load_dotenv
import pandas as pd
from datetime import datetime

load_dotenv()

FRED_API_KEY = os.getenv("FRED_API_KEY")
ECOS_API_KEY = os.getenv("ECOS_API_KEY")

# --- FRED 데이터 가져오기 (미국) ---
def get_fred_data(series_id, start_date="2015-01-01"):
    """FRED API에서 데이터 가져오기"""
    base_url = "https://api.stlouisfed.org/fred/series/observations"
    
    params = {
        "series_id": series_id,
        "api_key": FRED_API_KEY,
        "file_type": "json",
        "observation_start": start_date,
        "frequency": "m"
    }
    
    try:
        response = requests.get(base_url, params=params)
        response.raise_for_status()
        data = response.json()
        
        observations = data.get("observations", [])
        df = pd.DataFrame(observations)
        df = df[df['value'] != '.']
        df['date'] = pd.to_datetime(df['date'])
        df['value'] = pd.to_numeric(df['value'])
        
        return df[['date', 'value']]
    
    except Exception as e:
        print(f"FRED API 오류 ({series_id}): {e}")
        return None

# --- 한국은행 데이터 가져오기 ---
def get_ecos_data(start_date="201501"):
    """한국은행 API에서 기준금리 가져오기"""
    
    if not ECOS_API_KEY:
        print("⚠️ ECOS_API_KEY가 설정되지 않았습니다.")
        print("한국은행 API 키를 발급받아 .env 파일에 추가하세요.")
        print("발급 URL: https://ecos.bok.or.kr/")
        return None
    
    print(f"🔍 한국은행 API 호출 중...")
    
    base_url = "https://ecos.bok.or.kr/api/StatisticSearch"
    
    try:
        url = f"{base_url}/{ECOS_API_KEY}/json/kr/1/1000/722Y001/M/{start_date}/{datetime.now().strftime('%Y%m')}/0101000"
        response = requests.get(url)
        response.raise_for_status()
        data = response.json()
        
        print(f"✅ API 응답 받음")
        
        if 'StatisticSearch' in data:
            if 'row' in data['StatisticSearch']:
                rows = data['StatisticSearch']['row']
                print(f"✅ {len(rows)}개 데이터 수신")
                
                df = pd.DataFrame(rows)
                df['date'] = pd.to_datetime(df['TIME'], format='%Y%m')
                df['value'] = pd.to_numeric(df['DATA_VALUE'])
                
                return df[['date', 'value']]
            else:
                print(f"❌ 'row' 키가 없음")
        else:
            print(f"❌ API 오류 응답: {data}")
            
        return None
    
    except Exception as e:
        print(f"❌ 한국은행 API 오류: {e}")
        return None

def display_comparison():
    """한미 기준금리 비교 출력"""
    print("\n" + "="*80)
    print("📊 한국 vs 미국 기준금리 비교 (최근 10년, 월별)")
    print("="*80 + "\n")
    
    print("데이터 수집 중...\n")
    
    # 데이터 수집 (기준금리만)
    kr_rate = get_ecos_data("201501")
    us_fedfunds = get_fred_data("FEDFUNDS", "2015-01-01")
    
    # 데이터 병합
    merged = None
    
    if kr_rate is not None:
        print(f"✅ 한국 데이터 {len(kr_rate)}개 수집 완료")
        kr_rate = kr_rate.rename(columns={'value': '한국_기준금리'})
        merged = kr_rate
    else:
        print("❌ 한국 데이터 수집 실패")
    
    if us_fedfunds is not None:
        print(f"✅ 미국 데이터 {len(us_fedfunds)}개 수집 완료")
        us_fedfunds = us_fedfunds.rename(columns={'value': '미국_기준금리'})
        if merged is None:
            merged = us_fedfunds
        else:
            merged = pd.merge(merged, us_fedfunds, on='date', how='outer')
    else:
        print("❌ 미국 데이터 수집 실패")
    
    if merged is None:
        print("❌ 데이터를 가져오지 못했습니다.")
        return
    
    # 날짜순 정렬
    merged = merged.sort_values('date')
    merged['연월'] = merged['date'].dt.strftime('%Y년 %m월')
    
    print("\n" + "="*80)
    print("📅 전체 10년 기준금리 데이터 (2015년 1월 ~ 현재)")
    print("="*80 + "\n")
    
    print(f"{'연월':<15} {'한국 기준금리':<20} {'미국 기준금리':<20}")
    print("-" * 60)
    
    # 전체 데이터 출력
    for _, row in merged.iterrows():
        kr = f"{row.get('한국_기준금리', 0):.2f}%" if pd.notna(row.get('한국_기준금리')) else "N/A"
        us_ff = f"{row.get('미국_기준금리', 0):.2f}%" if pd.notna(row.get('미국_기준금리')) else "N/A"
        
        print(f"{row['연월']:<15} {kr:<20} {us_ff:<20}")
    
    # CSV 저장
    output_file = "한미_기준금리_비교_10년.csv"
    merged.to_csv(output_file, index=False, encoding='utf-8-sig')
    print(f"\n💾 전체 데이터가 '{output_file}'에 저장되었습니다.")
    
    # 통계 요약
    print("\n" + "="*80)
    print("📈 통계 요약 (최근 10년)")
    print("="*80)
    
    for col in ['한국_기준금리', '미국_기준금리']:
        if col in merged.columns:
            valid_data = merged[col].dropna()
            if len(valid_data) > 0:
                max_date = merged[merged[col] == valid_data.max()]['연월'].values[0]
                min_date = merged[merged[col] == valid_data.min()]['연월'].values[0]
                
                print(f"\n{col}:")
                print(f"  데이터 개수: {len(valid_data)}개월")
                print(f"  평균: {valid_data.mean():.2f}%")
                print(f"  최고: {valid_data.max():.2f}% ({max_date})")
                print(f"  최저: {valid_data.min():.2f}% ({min_date})")
                print(f"  현재: {valid_data.iloc[-1]:.2f}%")
    
    # 금리 차이 분석
    if '한국_기준금리' in merged.columns and '미국_기준금리' in merged.columns:
        merged['금리차이'] = merged['한국_기준금리'] - merged['미국_기준금리']
        valid_diff = merged['금리차이'].dropna()
        
        if len(valid_diff) > 0:
            print(f"\n💡 한미 금리 차이 (한국 - 미국):")
            print(f"  평균 차이: {valid_diff.mean():.2f}%p")
            print(f"  최대 차이: {valid_diff.max():.2f}%p ({merged[merged['금리차이'] == valid_diff.max()]['연월'].values[0]})")
            print(f"  최소 차이: {valid_diff.min():.2f}%p ({merged[merged['금리차이'] == valid_diff.min()]['연월'].values[0]})")
            print(f"  현재 차이: {valid_diff.iloc[-1]:.2f}%p")

if __name__ == "__main__":
    display_comparison()


📊 한국 vs 미국 기준금리 비교 (최근 10년, 월별)

데이터 수집 중...

🔍 한국은행 API 호출 중...
✅ API 응답 받음
✅ 129개 데이터 수신
✅ 한국 데이터 129개 수집 완료
✅ 미국 데이터 129개 수집 완료

📅 전체 10년 기준금리 데이터 (2015년 1월 ~ 현재)

연월              한국 기준금리              미국 기준금리             
------------------------------------------------------------
2015년 01월       2.00%                0.11%               
2015년 02월       2.00%                0.11%               
2015년 03월       1.75%                0.11%               
2015년 04월       1.75%                0.12%               
2015년 05월       1.75%                0.12%               
2015년 06월       1.50%                0.13%               
2015년 07월       1.50%                0.13%               
2015년 08월       1.50%                0.14%               
2015년 09월       1.50%                0.14%               
2015년 10월       1.50%                0.12%               
2015년 11월       1.50%                0.12%               
2015년 12월       1.50%                0.24%               
2016년 01월       1

AttributeError: 'str' object has no attribute 'sort_index'